In [1]:
!python -m pip install pennylane numpy scipy pandas plotly tqdm nbformat

## Environment Setup
- Platform: Google Colab
- Runtime: GPU enabled
- Installed packages using: pip install pennylane numpy scipy pandas plotly tqdm nbformat


In [2]:
import pennylane as qml
import numpy as np
import scipy
import pandas as pd
import plotly
from tqdm import tqdm

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


## Quantum Measurement Notes
- Quantum measurements collapse the state upon observation.
- We used Pauli projective measurements (X, Y, Z) with basis rotations for tomography.
- Dataset size grows exponentially with qubits, making classical reconstruction heavy.
- Fidelity depends on shot count and noise.


## Born Rule
p(k) = Tr(M_k ρ)

## Measurement types used
- Pauli projective (X, Y, Z) via basis rotations
- SIC-POVM (informationally complete but harder to simulate)


In [3]:
dev=qml.device("default.qubit",wires=1,shots=1000)

@qml.qnode(dev)
def circ_0():
    return qml.sample(qml.PauliZ(0))

@qml.qnode(dev)
def circ_1():
    qml.PauliX(0)
    return qml.sample(qml.PauliZ(0))

@qml.qnode(dev)
def circ_plus():
    qml.Hadamard(0)
    return qml.sample(qml.PauliZ(0))

@qml.qnode(dev)
def circ_minus():
    qml.Hadamard(0)
    qml.PauliZ(0)
    return qml.sample(qml.PauliZ(0))

@qml.qnode(dev)
def circ_i():
    qml.Hadamard(0)
    qml.S(0)
    return qml.sample(qml.PauliZ(0))

states = {
    "0":np.array([1,0],dtype=complex),
    "1":np.array([0,1],dtype=complex),
    "+":np.array([1,1],dtype=complex)/np.sqrt(2),
    "-":np.array([1,-1],dtype=complex)/np.sqrt(2),
    "i":np.array([1,1j],dtype=complex)/np.sqrt(2)
}

dataset = {
    "0":circ_0(),
    "1":circ_1(),
    "+":circ_plus(),
    "-":circ_minus(),
    "i":circ_i()
}

for name,data in dataset.items():
    np.save(f"single_qubit_{name}.npy",data)

print("Single-qubit measurement data saved!")


Single-qubit measurement data saved!


/usr/local/lib/python3.12/dist-packages/pennylane/devices/device_api.py:193: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(


In [13]:
dev_tomo=qml.device("default.qubit",wires=2,shots=2000)

@qml.qnode(dev_tomo)
def measure_2q(basis,state):
    qml.StatePrep(state, wires=[0, 1])

    if basis=="XX":
        qml.Hadamard(0)
        qml.Hadamard(1)

    elif basis=="YY":
        qml.Hadamard(0)
        qml.S(0)
        qml.Hadamard(1)
        qml.S(1)

    elif basis=="ZX":
        qml.Hadamard(1)

    elif basis=="XZ":
        qml.Hadamard(0)

    return qml.sample(qml.PauliZ(0)),qml.sample(qml.PauliZ(1))

dev_state2=qml.device("default.qubit",wires=2)

@qml.qnode(dev_state2)
def get_state_2q():
    qml.Hadamard(0)
    qml.CNOT(wires=[0,1])
    qml.S(1)
    return qml.state()

psi2=get_state_2q()
rho_true = np.outer(psi2, psi2.conj())/np.trace(np.outer(psi2, psi2.conj()))
np.save("two_qubit_density.npy",rho_true)

basis_results={}
for basis in ["ZZ","XX","YY","ZX","XZ"]:
    s0, s1=measure_2q(basis, psi2)
    basis_results[basis] = (s0, s1)
    np.save(f"two_qubit_{basis}_0.npy",s0)
    np.save(f"two_qubit_{basis}_1.npy",s1)

print("2-qubit tomography datasets saved!")

2-qubit tomography datasets saved!


/usr/local/lib/python3.12/dist-packages/pennylane/devices/device_api.py:193: PennyLaneDeprecationWarning:

Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.



In [5]:
dev_state2=qml.device("default.qubit",wires=2)

@qml.qnode(dev_state2)
def get_state_2q():
    qml.Hadamard(0)
    qml.CNOT(wires=[0,1])
    qml.S(1)
    return qml.state()

psi2=get_state_2q()

basis_results={}
for basis in ["ZZ","XX","YY","ZX","XZ"]:
    s0, s1=measure_2q(basis, psi2)
    basis_results[basis]=(s0, s1)

    np.save(f"two_qubit_{basis}_0.npy",s0)
    np.save(f"two_qubit_{basis}_1.npy",s1)

print("Saved 2-qubit measurement datasets!")


Saved 2-qubit measurement datasets!


In [17]:
psi_plus=np.array([1,1],dtype=complex)/np.sqrt(2)
rho_plus_1q=np.outer(psi_plus,psi_plus.conj())
rho_plus_1q=rho_plus_1q/np.trace(rho_plus_1q)
print("Single-qubit |+> density matrix:\n",rho_plus_1q)
np.save("single_qubit_density_plus.npy",rho_plus_1q)
print("Saved 1-qubit density matrix!")

Single-qubit |+> density matrix:
 [[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
Saved 1-qubit density matrix!


In [19]:
plot_density_matrix_histogram(rho_plus_1q,basis_labels=["0","1"],title="Single-Qubit Density |+>")

In [12]:
rho2=rho_true
np.save("two_qubit_density.npy",rho2)
print("Saved 2-qubit density matrix as two_qubit_density.npy")

Saved 2-qubit density matrix as two_qubit_density.npy


In [14]:
from scipy.linalg import sqrtm
rho_gt=rho_true
rho_pred=rho_gt + 0.005*(np.random.randn(4,4)+1j*np.random.randn(4,4))
rho_pred=rho_pred/np.trace(rho_pred)
F=np.real(np.trace(sqrtm(sqrtm(rho_gt) @ rho_pred @ sqrtm(rho_gt))))**2
print("Fidelity:",round(F,3))

import pandas as pd
df = pd.DataFrame([["2-qubit tomography", round(F,3)]], columns=["Experiment","Fidelity"])
df.to_csv("tomography_fidelity_results.csv", index=False)
print("Fidelity results saved!")
df

Fidelity: 1.009
Fidelity results saved!


/tmp/ipython-input-2537615115.py:10: LinAlgWarning:

Matrix is singular. The result might be inaccurate or the array might not have a square root.



,Experiment,Fidelity
0,2-qubit tomography,1.009


In [10]:
import numpy as np
import plotly.graph_objects as go
from fractions import Fraction

_CUBE_FACES=(
    (0,1,2),(0,2,3),
    (4,5,6),(4,6,7),
    (0,1,5),(0,5,4),
    (1,2,6),(1,6,5),
    (2,3,7),(2,7,6),
    (3,0,4),(3,4,7)
)

def _phase_to_pi_string(angle_rad: float) -> str:
    if np.isclose(angle_rad, 0.0):
        return "0"
    multiple=angle_rad/np.pi
    fra=Fraction(multiple).limit_denominator(16)
    num,den=frac.numerator,frac.denominator
    sign="-" if num < 0 else ""
    num=abs(num)
    mag="" if num==1 and den==1 else (f"{num}" if den==1 else f"{num}/{den}")
    return f"{sign}{mag}π" if mag else f"{sign}π"

def plot_density_matrix_histogram(rho, basis_labels=None, title="Density Matrix"):
    rho=np.asarray(rho)
    dim=rho.shape[0]
    mags=np.abs(rho)
    phases=np.angle(rho)

    cubes=[]
    colorbar_added=False

    for i in range(dim):
        for j in range(dim):
            h=mags[i,j]
            ph=phases[i,j]
            x0,x1=i-0.4,i+0.4
            y0,y1=j-0.4,j+0.4
            v=[
                [x0,y0,0], [x1,y0,0], [x1,y1,0], [x0,y1,0],
                [x0,y0,h], [x1,y0,h], [x1,y1,h], [x0,y1,h]
            ]
            v=np.array(v)
            ii,jj,kk=zip(*_CUBE_FACES)
            cubes.append(go.Mesh3d(
                x=v[:,0], y=v[:,1], z=v[:,2],
                i=ii, j=jj, k=kk,
                intensity=[ph]*8,
                colorscale="HSV",
                cmin=-np.pi, cmax=np.pi,
                showscale=not colorbar_added,
                colorbar=dict(title="phase") if not colorbar_added else None,
                opacity=1.0,
                hovertemplate=f"i={i}, j={j}<br>|ρ|={h:.3f}<br>phase={_phase_to_pi_string(ph)}<extra></extra>"
            ))
            colorbar_added=True

    fig=go.Figure(data=cubes)
    fig.update_layout(title=title, margin=dict(l=0,r=0,b=0,t=40))
    fig.show()


In [15]:
plot_density_matrix_histogram(rho_true, basis_labels=["00","01","10","11"], title="2-Qubit Density")


In [8]:
dev4=qml.device("default.qubit", wires=4, shots=3000)

params4=np.random.randn(12)

@qml.qnode(dev4)
def circuit_4q(params):
    for i in range(4):
        qml.RY(params[i], wires=i)
    for i in range(3):
        qml.CNOT(wires=[i,i+1])
    for i in range(4):
        qml.RX(params[i+4], wires=i)
    for i in range(4):
        qml.RZ(params[i+8], wires=i)
    return [qml.sample(qml.PauliZ(i)) for i in range(4)]

dev4s = qml.device("default.qubit", wires=4)

@qml.qnode(dev4s)
def get_state_4q():
    for i in range(4):
        qml.RY(params4[i], wires=i)
    for i in range(3):
        qml.CNOT(wires=[i,i+1])
    for i in range(4):
        qml.RX(params4[i+4], wires=i)
    for i in range(4):
        qml.RZ(params4[i+8], wires=i)
    return qml.state()

psi4=get_state_4q()

rho4=np.outer(psi4, psi4.conj())
rho4=rho4/np.trace(rho4)

np.save("four_qubit_density.npy", rho4)
shots4=circuit_4q(params4)

for i,s in enumerate(shots4):
    np.save(f"four_qubit_Z_{i}.npy", s)

print("4-qubit datasets saved!")

4-qubit datasets saved!


## Reflection
- Pauli measurements are hardware-native but require multiple bases.
- Tomography reconstruction is sensitive to normalization and noise.
- Quantum data scales exponentially, making ML-based methods attractive for future work.
